# Visualizing Example Cells

In [1]:
import os
from meshparty import trimesh_io, trimesh_vtk, meshwork
from caveclient import CAVEclient
import numpy as np
import cloudvolume
import itkwidgets

#setting up the cave client, if you have not done this before checkout the notebook CAVEsetup.ipynb at 
#https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/CAVEsetup.ipynb

#client = CAVEclient('minnie65_public_v117')
#client.auth.get_new_token()
#client.auth.save_token(token="c0ac8c614f75648f59f4f29130d3b56d")

dataset_name = 'minnie65_public_v117'
client = CAVEclient(dataset_name)
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)
# to enable to cache the mesh data, create a MeshMeta object:
"""
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_v117_meshes',
                         map_gs_to_https=True)
"""

#cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)

SSLError: HTTPSConnectionPool(host='minnie.microns-daf.com', port=443): Max retries exceeded with url: /segmentation/table/minnie65_public_v117/info (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))

## Find matching elements across different databases

*sometimes the same cell can correspond to different `root_id` on different databases*

1. Find the soma coordinates of the element of interest (e.g `864691135687245024`)
    ```
    import pandas
    em_data = pandas.read_csv('../data/em_connectomics_mesh_skeletons_directory_proof_status.csv')
    ID = 864691135687245024
    em_data['pt_position'][em_data['root_id']==ID]
    ```
2. go to neuroglancer: https://ngl.microns-explorer.org

    - go to that position (top left coordinates)
    - find the other `root_id` that corresponds to that object
   
3. Download the associated mesh from the precomputed mesh database:
    ```
    MC_ID = 864691135367179769
    cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)
    mesh = cv.mesh.get(MC_ID,lod=3)[MC_ID]
    ```


# Load cellular and synaptic data

In [2]:
voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm

# -------------------------
# --- Martinotti cell -----
# -------------------------
MC_ID = 864691135367179769
mesh_MC = cv.mesh.get(MC_ID,lod=3)[MC_ID]
poly_data1 = trimesh_vtk.trimesh_to_vtk(mesh_MC.vertices, mesh_MC.faces, None)
# --- Martinotti cell synapses
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([MC_ID]))
syn_pts_MC = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution


# -------------------------
# ---    Basket cell  -----
# -------------------------
BC_ID = 864691135269913253
mesh_BC = cv.mesh.get(BC_ID,lod=3)[BC_ID]
poly_data2 = trimesh_vtk.trimesh_to_vtk(mesh_BC.vertices, mesh_BC.faces, None)
# --- Basket cell synapses
post_synapse_BC = client.materialize.synapse_query(post_ids = np.array([BC_ID]))
syn_pts_BC = np.vstack(post_synapse_BC['ctr_pt_position'].values) * voxel_resolution

# ---------------------------------------
# A Layer 5 pyramidal cell for comparison
L5P_ID = 864691135627205572
mesh_L5P = cv.mesh.get(L5P_ID,lod=3)[L5P_ID]
poly_data3 = trimesh_vtk.trimesh_to_vtk(mesh_L5P.vertices, mesh3.faces, None)

SSLError: HTTPSConnectionPool(host='minnie.microns-daf.com', port=443): Max retries exceeded with url: /materialize/api/v2/datastack/minnie65_public_v117/versions (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))

# Showing both the Basket and Martinotti cells in the Cortical Column

In [ ]:
#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data2, poly_data1, poly_data3])
viewer

# Basket cell with synapses

In [8]:
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([BC_ID]))
poly_data = trimesh_vtk.trimesh_to_vtk(mesh_BC.vertices, mesh_BC.faces, None)

syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerBC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerBC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Martinotti cell with synapses

In [3]:
poly_data = trimesh_vtk.trimesh_to_vtk(mesh_MC.vertices, mesh_MC.faces, None)
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([MC_ID]))
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerMC =itkwidgets.view(geometries=[poly_data_MC],
                          point_sets = [syn_pts_MC],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerMC

NameError: name 'voxel_resolution' is not defined

# More controlled rendering with `trimesh_vtk`

### 1) Choose your camera angle ! 
    --> press `"k"` before closing

In [ ]:
# --- Martinotti cell
mesh_actors_MC = trimesh_vtk.mesh_actor(mesh_MC)
# --- Basket cell
mesh_actors_BC = trimesh_vtk.mesh_actor(mesh_BC)
# --- L5 Pyr cell
mesh_actors_L5P = trimesh_vtk.mesh_actor(mesh_L5P)

#
#   PRESS "k" when you are happy with the view !
#
K = trimesh_vtk.render_actors([mesh_actors_MC, mesh_actors_BC, mesh_actors_L5P],
                              return_keyframes=True)
Camera = K[-1][0] # we store the last position as the Camera !

### 2) Screenshot the the full view

In [86]:
orange=(1., 0.5, 0.2)
red = (1., 0.1, 0.1)
green = (0., 0.5, 0.)

mesh_actors_MC = trimesh_vtk.mesh_actor(mesh_MC, opacity=0.8, color=orange)
mesh_actors_BC = trimesh_vtk.mesh_actor(mesh_BC, opacity=0.6, color=red)
mesh_actors_L5P = trimesh_vtk.mesh_actor(mesh_L5P, opacity=0.4, color=green)

trimesh_vtk.render_actors([mesh_actors_MC, mesh_actors_BC, mesh_actors_L5P,
                               trimesh_vtk.scale_bar_actor(np.array(Camera.GetFocalPoint())-300e3, Camera,
                                                           length=10e3*4,
                                                           linewidth=1)],
                          camera=Camera)

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x5596ca59d480) at 0x7f62d94fb2e0>

### 3) Screenshot the Martinotti cell with synapses

### 4) Screenshot the Zoomed Dendritic Branches in the Martinotti cell

In [ ]:
zoom_factor = 20.

mesh_actors_MC = trimesh_vtk.mesh_actor(mesh_MC, opacity=0.5, face_colors=orange, color=purple)
syn_actors_MC = trimesh_vtk.point_cloud_actor(syn_pts_MC, size=5e2, color=cyan)

Camera.Zoom(zoom_factor)
trimesh_vtk.render_actors([mesh_actors_MC, syn_actors_MC],camera=Camera)
Camera.Zoom(1./zoom_factor)

### 5) Screenshot the Basket cell with synapses

### 6) Screenshot the Zoomed Dendritic Branches in the Basket cell

In [94]:
zoom_factor = 20.

mesh_actors_BC = trimesh_vtk.mesh_actor(mesh_BC, opacity=0.5, face_colors=red, color=red)
syn_actors_BC = trimesh_vtk.point_cloud_actor(syn_pts_BC, size=5e2, color=cyan)

Camera.Zoom(zoom_factor)
trimesh_vtk.render_actors([mesh_actors_BC, syn_actors_BC],camera=Camera)
Camera.Zoom(1./zoom_factor)

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


In [12]:
trimesh_vtk.oriented_camera?

In [ ]:
trimesh_vtk.render_actors([mesh_actors_MC, mesh_actors_BC])

In [11]:
trimesh_vtk.camera_from_quat?

In [21]:
trimesh_vtk.render_actors?

In [74]:
trimesh_vtk.scale_bar_actor?

In [60]:
Camera.Zoom(4)

In [61]:
Camera.Zoom(0.25)